In [1]:
from src.model.pl_modules import RacingF1Detector
from src.data.datasets import RacingF1Dataset
from src.data.pl_data_modules import RacingF1DataModule
from src.data.custom_transforms import CustomRescale
from pytorch_lightning import Trainer
from src.utils import draw_bounding_box, tensor_to_image
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from torch.utils.data import DataLoader
from torchvision import transforms

import torch
import torchvision
import pytorch_lightning as pl

## Training

In [ ]:
pl.seed_everything(42, workers=True)

hparams = {
	"lr": 0.005,
	"momentum": 0.9,
	"weight_decay": 0.0005,
	"batch_size": 32
}

detector = RacingF1Detector(hparams)

In [2]:
augmentation = transforms.Compose([
	CustomRescale((350, 350)),
])

data_module = RacingF1DataModule.from_directories(hparams["batch_size"], "dataset",
                                                  data_transform=augmentation)

data_module.setup()

In [7]:
logger = TensorBoardLogger('tb_logs', name='racing_detector')

In [8]:
%load_ext tensorboard
%tensorboard --logdir tb_logs/

In [9]:
early_stop_callback = EarlyStopping(
	monitor="val_loss",
	patience=3,
	verbose=False,
	mode="min",
)

checkpoint_callback = ModelCheckpoint(
    dirpath="./savedcheckpoints",
    filename="{epoch}{val_loss:.3f}",
    monitor="val_loss",
    save_top_k=2,
    save_last=True,
    mode="min",
)

trainer = Trainer(gpus=0, max_epochs=1, logger=logger, callbacks=[early_stop_callback, checkpoint_callback])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [11]:
trainer.fit(detector, datamodule=data_module)


  | Name     | Type       | Params
----------------------------------------
0 | model    | FasterRCNN | 41.3 M
1 | test_mAP | MAP        | 0     
----------------------------------------
41.1 M    Trainable params
222 K     Non-trainable params
41.3 M    Total params
165.197   Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

## Testing

In [ ]:
trainer = pl.Trainer(gpus=1)

trainer.test(model=model, dataloaders=test_dataloader)